In [1]:
from sympy import symbols, sin, cos, tan
from sympy.physics.mechanics import *
q1, q2, q3, u1, u2, u3=dynamicsymbols('q1 q2 q3 u1 u2 u3')
q1d, q2d, q3d, u1d, u2d, u3d=dynamicsymbols('q1 q2 q3 u1 u2 u3', 1)
r, m, g=symbols('r m g')
mechanics_printing(pretty_print=True, use_unicode=True)

N=ReferenceFrame('N')
Y=N.orientnew('Y', 'Axis', [q1, N.x])
L=Y.orientnew('L', 'Axis', [q2, Y.y])
R=L.orientnew('R', 'Axis', [q3, L.z])
w_R_N_qd=R.ang_vel_in(N)
R.set_ang_vel(N, u1*L.x+u2*L.y+u3*L.z)


In [2]:
C=Point('C')
C.set_vel(N, 0)
Dmc=C.locatenew('Dmc', r*L.z)
mprint(Dmc.v2pt_theory(C, N, R))
Dmc.v2pt_theory(C, N, R)

r*u2*L.x - r*u1*L.y


r⋅u₂ l_x + -r⋅u₁ l_y

In [3]:
I=inertia(L, m/4*r**2, m/2*r**2, m/4*r**2)
mprint(I)
I

m*r**2/4*(L.x|L.x) + m*r**2/2*(L.y|L.y) + m*r**2/4*(L.z|L.z)


   2
m⋅r 
────
 4   l_x⊗l_x +    2
m⋅r 
────
 2   l_y⊗l_y +    2
m⋅r 
────
 4   l_z⊗l_z

In [4]:
kd=[dot(R.ang_vel_in(N)-w_R_N_qd, uv) for uv in L]
mprint(kd)
kd

[u1 - cos(q2)*q1', u2 - q2', u3 - sin(q2)*q1' - q3']


[u₁ - cos(q₂)⋅q₁̇, u₂ - q₂̇, u₃ - sin(q₂)⋅q₁̇ - q₃̇]

In [5]:
ForceList=[(Dmc, -m*g*Y.z)]
BodyD=RigidBody('BodyD', Dmc, R, m, (I, Dmc))
BodyList=[BodyD]

In [6]:
KM=KanesMethod(N, q_ind=[q1, q2, q3], u_ind=[u1, u2, u3], kd_eqs=kd)
(fr, frstar)=KM.kanes_equations(BodyList, ForceList)
MM=KM.mass_matrix
forcing=KM.forcing
rhs=MM.inv()*forcing
kdd=KM.kindiffdict()
rhs=rhs.subs(kdd)
rhs.simplify()
mprint(rhs)
rhs

Matrix([
[                         (6*u1*tan(q2) - u3)*u2/5],
[(4*g*sin(q2) - 5*r*u1**2*tan(q2) + r*u1*u3)/(6*r)],
[                                           -u1*u2]])


⎡        (6⋅u₁⋅tan(q₂) - u₃)⋅u₂         ⎤
⎢        ──────────────────────         ⎥
⎢                  5                    ⎥
⎢                                       ⎥
⎢                    2                  ⎥
⎢4⋅g⋅sin(q₂) - 5⋅r⋅u₁ ⋅tan(q₂) + r⋅u₁⋅u₃⎥
⎢───────────────────────────────────────⎥
⎢                  6⋅r                  ⎥
⎢                                       ⎥
⎣                -u₁⋅u₂                 ⎦